# HMM и Разметка частей речи

Мы не будем реализовывать алгоритмы HMM с нуля, а воспользуемся библиотекой ``hmmlearn`` 

In [1]:
from hmmlearn.hmm import MultinomialHMM
import numpy as np
import nltk
import pandas as pd
from nltk.tokenize import wordpunct_tokenize
from collections import Counter
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split

## Строим POS-tagger для английского языка
- для обучения мы воспользуемся размеченными данными [Brown Corpus](https://en.wikipedia.org/wiki/Brown_Corpus), который можно скачать прямо из библитеки nltk
- мы будем использовать universal_tagset из nltk (это не таги Universal Dependencies, о которых мы говорили на лекции)

In [2]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /home/variya/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/variya/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### Вам нужно выполнить следующие шаги, чтобы построить POS-tagger

**Вопросы:**

1. Возьмите размеченные предложения из  ``nltk.corpus.brown.tagged_sents(tagset='universal')``  

    - Используйте ``sklearn.model_selection.train_test_split`` чтобы разделить корпус на  80% training data и 20% testing data.

2. Создайте  array ``pos_tags``  содержащий все уникальные POS таги, которые есть в трейн корпусе. Сколько уникальных тагов у вас получилось?

3. Найдите 5000 наиболее частов токенов (используйте collections.Counter). 
    - Перед подсчетом сделайте все буквы в словах маленькими (lowercase)
    - Сохраните эти 5000 унаиболее частотных токенов в array ``vocab``. 
    -  Добавьте токен '[UNK]' в качестве первого элемента ``vocab`` для обозначения всех слов, не вошедших вчастотный словарь =  "out of vocabulary")
    - Проверьте себя: первые 5 элементов ``vocab`` должны быть \["\[UNK\]", "the", ",", ".", "of", ...\]

4. Используйте ``hmmlearn.hmm.MultinomialHMM``, чтобы создать модель``pos_model`` для разметки частей речи (POS-tagging). 

  * Установите ``pos_model.startprob_`` используя информацию о доле предложений, начинающихся с каждого из POS тагов из вашего списка уникальных тагов. Например, какая доля предложений начинается с глагола и т.д. 
     -  Подсказка: это должен быть лист длины ``len(pos_tags)``, сумма вероятностей бытьв начале предложения должна бытьравна 1.
  
  * Установите ``pos_model.transmat_`` - вероятность перехода от одного тага к другому на основе данных трейн корпуса.
      - Подсказка: это должна быть матрица ``(len(pos_tags), len(pos_tags))``, сумма по каждой из строк матрицы должна быть равна 1.
  
  * Установите ``pos_model.emissionprob_`` - вероятность для каждого токена из ``vocab`` относится к какой-либо части речи. 
      - Убедитесь, что все токены состоят только из маленьких букв. 
      - Все токены не из ``vocab`` заменены на  "\[UNK\]". 
      - Подсказка: это должна быть матрица ``(len(pos_tags), len(vocab))`` , сумма по каждой из строк матрицы должна быть равна 1.
  
5. Напишите функцию``get_pos(sentence)``, которая возвращает наиболее вероятную последовательность тегов для некоторого предложения that (``sentence``) 
    - в этой функции используйте pos_model.decode(...). 

6. Попробуйте применить вашу модель к нескольким предложениям (слова только измаленьких букв, без пунктуации).
    - Обязательно попробуйте для предложений: "this is a test", "saint petersburg is the second-largest city in russia.", "i know how to use hmm". 
    - Прокомментируйте получившиеся результаты. Похоже на правду?
    - Если появились идеи, почему модель ошибается, напишите.

**Бонус:** Оцените правильность этой модели на тестовых данных

In [3]:
class TextHMM(BaseEstimator):

    def __init__(self, k, lower=True):
        self.pos_tags_ = None
        self.vocab_ = None
        self.k = k
        self.none_token = '[UNK]'
        self.lower = lower
        self.model = None

    def __transform(self, token, lower):
        trans_token = token
        if lower:
            trans_token = token.lower()
        if self.vocab_:
            if trans_token not in self.vocab_:
                return self.none_token
        return trans_token

    def _set_pos_tags(self, taged_sentences):
        sents_pos_tags = [pos_tag[1] for sent in taged_sentences
                          for pos_tag in sent]
        self.pos_tags_ = list(set(sents_pos_tags))

    def _set_vocab(self, tokens):
        most_common_tokens = Counter(tokens).most_common(self.k)
        self.vocab_ = [token[0] for token in most_common_tokens]
        self.vocab_.insert(0, self.none_token)

    def _get_startprob(self, taged_sentences):
        first_pos_tags = Counter([sent[0][1] for sent in
                                 taged_sentences])
        total_sents = len(taged_sentences)
        startprob = np.array([first_pos_tags[pos_tag] / total_sents
                             for pos_tag in self.pos_tags_])
        assert abs(1 - startprob.sum()) < 1e-5
        return startprob

    def _get_transmat(self, taged_sentences):
        transmat = np.zeros((len(self.pos_tags_), len(self.pos_tags_)))
        for sentence in taged_sentences:
            for k in range(len(sentence) - 1):
                i = self.pos_tags_.index(sentence[k][1])
                j = self.pos_tags_.index(sentence[k + 1][1])
                transmat[i, j] += 1
        transmat /= transmat.sum(axis=1)[:, np.newaxis]
        assert all(np.abs(1 - transmat.sum(axis=1)) < 1e-5)
        return transmat

    def _get_emissionprob(self, taged_sentences):
        pair_count = Counter()
        for taged_sentence in taged_sentences:
            pair_count.update(taged_sentence)
        emissionprob = np.zeros((len(self.pos_tags_), len(self.vocab_)))
        for (i, pos_tag) in enumerate(self.pos_tags_):
            for (j, token) in enumerate(self.vocab_):
                emissionprob[i, j] = pair_count[(token, pos_tag)]
        emissionprob /= emissionprob.sum(axis=1)[:, np.newaxis]
        assert all(np.abs(1 - emissionprob.sum(axis=1)) < 1e-5)
        return emissionprob

    def fit(self, taged_sentences, y=None):
        self._set_pos_tags(taged_sentences)
        tokens = [self.__transform(pos_tag[0], self.lower) for sent in
                  taged_sentences for pos_tag in sent]
        self._set_vocab(tokens)

        transform_pair = lambda pair: (self.__transform(pair[0],
                self.lower), pair[1])
        transform_sentance = lambda sent: [transform_pair(pair)
                for pair in sent]
        transformed_taged_sentences = [transform_sentance(sent)
                for sent in taged_sentences]

        # MultinomialHMM model

        self.model = MultinomialHMM(n_components=len(self.pos_tags_))
        self.model.emissionprob_ = \
            self._get_emissionprob(transformed_taged_sentences)
        self.model.startprob_ = \
            self._get_startprob(transformed_taged_sentences)
        self.model.transmat_ = \
            self._get_transmat(transformed_taged_sentences)

        return self.model

    def __get_X(self, tokens):
        token_code = [self.vocab_.index(self.__transform(token,
                      self.lower)) for token in tokens]
        return np.array(token_code)[:, np.newaxis]

    def get_pos(self, sentence):
        if self.lower:
            sentence = sentence.lower()
        X = self.__get_X(sentence.split(' '))
        (logprob, seq) = self.model.decode(X)
        return [self.pos_tags_[s] for s in seq]

In [4]:
sents = nltk.corpus.brown.tagged_sents(tagset='universal')
train_sents, test_sents = train_test_split(sents, test_size=0.2)

In [5]:
text_hmm = TextHMM(k=5000, lower=True)

In [6]:
text_hmm.fit(train_sents)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=12,
        n_iter=10, params='ste', random_state=None, startprob_prior=1.0,
        tol=0.01, transmat_prior=1.0, verbose=False)

In [7]:
text_hmm.get_pos('this is a test')

['DET', 'VERB', 'DET', 'NOUN']

Выглядит верно

In [8]:
text_hmm.get_pos('saint petersburg is the second-largest city in russia.')

['NOUN', 'NOUN', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN']

Название города не распознал, но saint отнес к существительному, в целом выглядит хорошо

In [9]:
text_hmm.get_pos('i know how to use hmm')

['PRON', 'VERB', 'ADV', 'PRT', 'VERB', 'VERB']

Здесь кажется несколько странным сочетания дух подряд идущих глаголов, но на самом деле такое сочетание популярно в обущающих данных и это повлияло на ошибочноую резметку слова hmm

In [10]:
text_hmm.model.transmat_[text_hmm.pos_tags_.index('VERB'), text_hmm.pos_tags_.index('VERB')]

0.18459706232631998

In [11]:
pd.DataFrame(text_hmm.model.transmat_, columns=text_hmm.pos_tags_, index=text_hmm.pos_tags_).loc['VERB'].sort_values(ascending=False)

VERB    0.184597
ADP     0.169327
DET     0.162736
ADV     0.103072
NOUN    0.097836
.       0.080964
PRT     0.065352
ADJ     0.057652
PRON    0.054790
CONJ    0.014490
NUM     0.009001
X       0.000185
Name: VERB, dtype: float64

In [12]:
text_hmm.get_pos('i like cats')

['PRON', 'ADP', 'NOUN']

In [13]:
text_hmm.get_pos('moscow')

['NOUN']

In [14]:
text_hmm.get_pos('the')

['DET']

In [15]:
# VERB - verbs (all tenses and modes)
# NOUN - nouns (common and proper)
# PRON - pronouns
# ADJ - adjectives
# ADV - adverbs
# ADP - adpositions (prepositions and postpositions)
# CONJ - conjunctions
# DET - determiners
# NUM - cardinal numbers
# PRT - particles or other function words
# X - other: foreign words, typos, abbreviations
# . - punctuation

# Оценка качества

In [16]:
from sklearn.metrics import accuracy_score, classification_report

In [17]:
test_text = ' '.join([pair[0] for sent in test_sents for pair in sent])
test_tags = [pair[1] for sent in test_sents for pair in sent]

In [18]:
pred_test_tags = text_hmm.get_pos(test_text)

In [19]:
assert len(pred_test_tags) == len(test_tags)

In [20]:
print('Accuracy {:.3f}'.format(accuracy_score(test_tags, pred_test_tags)))

Accuracy 0.928


In [21]:
print(classification_report(test_tags, pred_test_tags))

             precision    recall  f1-score   support

          .       1.00      1.00      1.00     29287
        ADJ       0.74      0.86      0.80     16449
        ADP       0.96      0.97      0.96     28678
        ADV       0.90      0.81      0.85     11254
       CONJ       0.99      0.99      0.99      7576
        DET       0.99      0.99      0.99     27130
       NOUN       0.89      0.92      0.90     54933
        NUM       0.96      0.76      0.85      2853
       PRON       0.98      0.98      0.98     10003
        PRT       0.90      0.88      0.89      6088
       VERB       0.94      0.88      0.91     36566
          X       0.21      0.41      0.27       231

avg / total       0.93      0.93      0.93    231048



Решение присылйте на почту  bdt-mf-ml-nlp-2020-q4@bigdatateam.org  

В теме письма укажите: ``HW2:CompLing. ФИО``

Пожалуйста, оставьте обратную связь о задании [по ссылке](http://rebrand.ly/mfnlp2020q4_feedback_hw02). Она (при желании) анонимна ;)